In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import nept

from loading_data import get_data

In [ ]:
import info.r066d3 as info

In [ ]:
home = os.path.expanduser("~")
emi_shortcut = os.path.join(home, "code", "emi_shortcut")
pickle_filepath = os.path.join(emi_shortcut, "cache", "pickled")
# pickle_filepath = 'E:/code/emi_shortcut/cache/pickled'
output_filepath = os.path.join(emi_shortcut, "plots")

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)

In [2]:
nept.Epoch([[0.0, 1.0], [0.5]])

ValueError: must have the same number of start and stop times

In [3]:
times = np.array([0.0, 1.0])

epoch = nept.Epoch(times)

In [5]:
i = epoch.expand(1.)

In [7]:
i.durations

array([ 3.])

In [ ]:
times[:, 0].shape[0]

In [ ]:
a = np.squeeze(times).astype(float)

In [ ]:
times.ndim

In [ ]:
a.ndim

In [ ]:
t = a[..., np.newaxis]

In [ ]:
t.ndim

In [ ]:
np.hstack((t[0][..., np.newaxis], t[1][..., np.newaxis]))

In [ ]:
a.spikes

In [ ]:
a.tuning_curves

In [ ]:
type(spikes[1:2]) == np.ndarray

# Saving many tuning curves on the same plot

In [ ]:
import matplotlib.gridspec as gridspec

rows = 8
cols = 14
fig = plt.figure(figsize=(10, 10))

grids = gridspec.GridSpec(cols, rows)
grids.update(wspace=0.02, hspace=0.05)

xx, yy = np.meshgrid(xedges, yedges)
for i in range(rows*cols):
    ax = plt.subplot(grids[i])
    pp = ax.pcolormesh(xx, yy, tuning_curves[i], cmap='pink_r')
    ax.axis('off')
    
plt.show()

# savepath = 'E:/code/emi_shortcut/plots/tuning/' + info.session_id + 'many_tuning-curves.png'
# plt.savefig(savepath, dpi=300, bbox_inches='tight')

# Testing Neuralynx events

In [ ]:
dataloc = 'C:/Users/EmilyWork/Desktop/20170104'
session = '2017-01-05_test1'
event_filename = 'Events.nev'
# event_labels = dict(on='on',
#                     off='off')
event_labels = dict(on='TTL Output on AcqSystem1_0 board 0 port 3 value (0x0002).',
                    off='TTL Output on AcqSystem1_0 board 0 port 3 value (0x0000).')

In [ ]:
filepath = os.path.join(dataloc, session, event_filename)
events = nept.load_events(filepath, event_labels)

In [ ]:
len(events['off']), len(events['on'])